# INPUT

In [ ]:
!pip install -r requirements.txt

In [97]:
###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 모델링 시작

In [98]:
#    Write codes...
# ... (Data preprocessing code here)

#데이터셋 로드
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')


In [99]:
# 모델링 준비
import math
import numpy as np
#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# 파라미터 튜닝
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# 대충 돌릴때 체크 : cross-val
from sklearn.model_selection import cross_val_score

# 평가
from sklearn.metrics import accuracy_score

In [100]:

# Split the data into training and testing sets

# 데이터 분할 (훈련 세트와 테스트 세트)
#X = input_data[['ec', 'cunt', 'HeatingEnergyUsage_cumsum', 'hvstCo', 'frtstCo', 'HeatingEnergyUsage_cumsum']] # 초기에 진행했던 데이터


X = input_data[['ec', 'cunt', 'HeatingEnergyUsage_cumsum', 'hvstCo', 'frtstCo', 'HeatingEnergyUsage_cumsum', 'inTp', 'inHd', 'inCo2', 'acSlrdQy',
               'grwtLt', 'frtstSetCo', 'hvstCo', 'FertilizerUsage', 'CO2Usage', 'MistUsageTime']]

#X = input_data[input_data.drop(columns=['outtrn_cumsum', 'frmDist']).columns]  # 세번째 데이터 - object타입 빼고 다 때려박기
#X = input_data[input_data.drop(columns=['Mist Cost', 'MistUsageTime', 'CO2Cost', 'CO2Usage', 'FertilizerCost', 'FertilizerUsage', 'WaterCost', 'WaterUsage', 'frtstJnt', 'hvstCo', 'flwrCo', 'hvstJnt', 'tcdmt', 'flanJnt', 'pllnLt', 'lefstalklt','lefstalklt','outWs','daysuplyqy', 'outtrn_cumsum', 'frmDist']).columns]
  #네번째 데이터 - 0 많은 컬럼 삭제

y = input_data[['outtrn_cumsum']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 현재최대 - modeling 파일 참고

(84840, 27)

(67872, 27) (67872, 1)
(16968, 27) (16968, 1)


In [101]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 로드 또는 생성 (X와 y를 사용자 데이터로 대체)
# X, y = ... 설정완료


# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
'''
최적 하이퍼파라미터: {'activation': 'relu', 'alpha': 0.0016601864044243652, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.01099749158180029, 'max_iter': 558}
최고 성능 (평가 지표): 763837967.5501446
'''
# Min-Max 스케일러 초기화 및 적용
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# ML` P 모델 초기화 및 설정
mlp = MLPRegressor(
    activation='relu',  # 'activation' 하이퍼파라미터 설정
    alpha=0.0016601864044243652,  # 'alpha' 하이퍼파라미터 설정
    hidden_layer_sizes=(100, 50, 25),  # 'hidden_layer_sizes' 하이퍼파라미터 설정
    learning_rate_init=0.01099749158180029,  # 'learning_rate_init' 하이퍼파라미터 설정
    max_iter=558  # 'max_iter' 하이퍼파라미터 설정
)

# 모델 학습
mlp.fit(X_train_normalized, y_train)

# 학습된 모델을 사용하여 예측
deep_predictions = mlp.predict(X_test_normalized)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [102]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions)) # 가장 큰 값 리턴
    #from sklearn.metrics import mean_squared_error
    #return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    return r2_score(y_test, y_pred) # 가장 큰값 리턴
    #from sklearn.metrics import r2_score
    #return r2_score(y_test, y_pred)

rmse = calculate_rmse(y_test, deep_predictions)
r2score = calculate_R2_score(y_test, deep_predictions)
# RMSE와 R^2 스코어 평가



In [103]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 27696.090852624304
R2_score: 0.4949029969452119


In [95]:
'''
#random search로 최적화하기
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# MLP 모델 초기화
mlp = MLPRegressor()

# 탐색할 하이퍼파라미터 범위 설정
param_dist = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],
    'activation': ['relu', 'tanh', 'logistic'],
    'alpha': uniform(0.0001, 0.01),  # L2 정규화 파라미터
    'learning_rate_init': uniform(0.001, 0.1),
    'max_iter': randint(100, 1000),
}

# 랜덤 서치 설정
random_search = RandomizedSearchCV(
    mlp, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42) #n_iter 수만큼 랜덤으로 선정해서 고르는거

# 랜덤 서치 실행
random_search.fit(X_train_normalized, y_train)
#random_search.fit(X,y) - 수정함

# 최적 하이퍼파라미터와 모델의 성능 출력
print("최적 하이퍼파라미터:", random_search.best_params_)
print("최고 성능 (평가 지표):", -random_search.best_score_)

# 최적 모델 가져오기
best_mlp = random_search.best_estimator_

# 04:04 실행 시작 04:31 종료 - 1트
'''
'''
최적 하이퍼파라미터: {'activation': 'relu', 'alpha': 0.0016601864044243652, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.01099749158180029, 'max_iter': 558}
최고 성능 (평가 지표): 763837967.5501446

'''

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


최적 하이퍼파라미터: {'activation': 'relu', 'alpha': 0.0016601864044243652, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.01099749158180029, 'max_iter': 558}
최고 성능 (평가 지표): 763837967.5501446


# 여기부턴 sequential 인듯

In [ ]:
# MLP 모델 아키텍처 정의

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_model(num_input=1):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=num_input))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

model = build_model(num_input=27)  #num_input에 input 피처의 개수를 넣어줘야 함

# 미니 배치 학습
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2)

# 모델 평가
model.evaluate(X_test, y_test)

#시간 01:27 실행시작 - 01:37 종료

In [ ]:
# 교차 검증
model = build_model(num_input=27)  #num_input에 input 피처의 개수를 넣어줘야 함
history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_split=0.25, verbose=2)

#
import matplotlib.pyplot as plt

def plot_loss_curve(total_epoch=10, start=1):
    plt.figure(figsize=(15, 5))
    plt.plot(range(start, total_epoch +1), history.history['loss'][start-1:total_epoch], label='Train')
    plt.plot(range(start, total_epoch +1), history.history['val_loss'][start-1:total_epoch], label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.legend()
    plt.show()

plot_loss_curve(total_epoch=200, start=1)

plot_loss_curve(total_epoch=200, start=20)
# 시간 01:37 실행시작 01:45 종료

In [83]:
#예측하기
deep_predictions = model.predict(X_test)

531/531 [==============================] - 1s 1ms/step


In [84]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions)) # 가장 큰 값 리턴
    #from sklearn.metrics import mean_squared_error
    #return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    return r2_score(y_test, y_pred) # 가장 큰값 리턴
    #from sklearn.metrics import r2_score
    #return r2_score(y_test, y_pred)

rmse = calculate_rmse(y_test, deep_predictions)
r2score = calculate_R2_score(y_test, deep_predictions)
# RMSE와 R^2 스코어 평가



In [85]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 36649.15888465624
R2_score: 0.12260355425253289


In [ ]:
#딥러닝 테스트

# 랜덤 시드 고정
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 데이터 전처리
from sklearn import datasets

#housing = datasets.load_boston()
#X_data = housing.data
#y_data = housing.target
X_data = X
y_data = y
X_data.shape
#덮어쓰기

In [ ]:
# 피처 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_data_scaled = scaler.fit_transform(X_data)

X_data_scaled[0]

# 학습 - 테스트 데이터셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=True, random_state = SEED)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

